In [1]:
# start my homework
import pandas as pd 
import numpy as np 
import cv2 # Used to manipulated the images 
np.random.seed(1337)

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.layers import concatenate as keras_concat
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df_train = pd.read_json('J:/kaggle/iceberg/train.json') #read the data

In [3]:
def get_scaled_imgs(df): #reshape the data and create a new image channel
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 - band_2

        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)

In [4]:
Xtrain = get_scaled_imgs(df_train) 
Ytrain = np.array(df_train['is_iceberg'])

In [5]:
#train only with (inc_angle not NA)
idx_tr = np.where(df_train.inc_angle!="na")
Ytrain = Ytrain[idx_tr[0]]
Xtrain = Xtrain[idx_tr[0],...]

In [6]:
def get_more_images(imgs): #flip, increase samples
    
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        a=imgs[i,:,:,0]
        b=imgs[i,:,:,1]
        c=imgs[i,:,:,2]
        
        av=cv2.flip(a,1)
        ah=cv2.flip(a,0)
        bv=cv2.flip(b,1)
        bh=cv2.flip(b,0)
        cv=cv2.flip(c,1)
        ch=cv2.flip(c,0)
        
        vert_flip_imgs.append(np.dstack((av, bv, cv)))
        hori_flip_imgs.append(np.dstack((ah, bh, ch)))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h))
    
    return more_images

In [7]:
Xtr_more = get_more_images(Xtrain)
Ytr_more = np.concatenate((Ytrain,Ytrain,Ytrain))

In [8]:
def getModel_inception():
    #google incepyion with residual learning
    input_img = Input(shape=(75, 75, 3))
    tower_1 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
    tower_1 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1)

    tower_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
    tower_2 = Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2)

    tower_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_img)
    tower_3 = Conv2D(64, (1, 1), padding='same', activation='relu')(tower_3)

    x = keras_concat([input_img, tower_1, tower_2, tower_3], axis=-1)

    #add Conv and MaxPooling layers to reduce parameters
    x = Conv2D(64, (3, 3), padding='valid', activation='relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    x = Conv2D(64, (3, 3), padding='valid', activation='relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    x = Conv2D(64, (3, 3), padding='valid', activation='relu')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # You must flatten the data for the dense layers
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    # output
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input_img, outputs=outputs)
    model.compile(optimizer=Adam(lr=0.001,decay=0.0),loss='binary_crossentropy',metrics=['accuracy'])
        
    return model

In [9]:
# fit getModel_inception to be an example
model = getModel_inception()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 75, 75, 64)   256         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 75, 75, 64)   256         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 75, 75, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (

In [10]:
batch_size = 32
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('model_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

In [11]:
model.fit(Xtr_more, Ytr_more, batch_size=batch_size, epochs=50, verbose=1, callbacks=[earlyStopping, mcp_save, reduce_lr_loss], validation_split=0.25)

Train on 3309 samples, validate on 1104 samples
Epoch 1/50
3309/3309 [==============================] - 831s 251ms/step - loss: 0.5517 - acc: 0.6969 - val_loss: 0.4110 - val_acc: 0.8025
Epoch 2/50
3309/3309 [==============================] - 815s 246ms/step - loss: 0.3597 - acc: 0.8447 - val_loss: 0.5139 - val_acc: 0.7690
Epoch 3/50
3309/3309 [==============================] - 813s 246ms/step - loss: 0.2997 - acc: 0.8631 - val_loss: 0.3086 - val_acc: 0.8569
Epoch 4/50
3309/3309 [==============================] - 813s 246ms/step - loss: 0.2561 - acc: 0.8852 - val_loss: 0.2853 - val_acc: 0.8750
Epoch 5/50
3309/3309 [==============================] - 813s 246ms/step - loss: 0.2269 - acc: 0.9000 - val_loss: 0.2617 - val_acc: 0.8850
Epoch 6/50
3309/3309 [==============================] - 814s 246ms/step - loss: 0.1841 - acc: 0.9217 - val_loss: 0.2570 - val_acc: 0.8976
Epoch 7/50
3309/3309 [==============================] - 812s 245ms/step - loss: 0.1870 - acc: 0.9238 - val_loss: 0.2518 - va

In [12]:
model.load_weights(filepath = 'model_wts.hdf5')

In [13]:
score = model.evaluate(Xtrain, Ytrain, verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])

1471/1471 [==============================] - 109s 74ms/step
Train score: 0.13245033653596078
Train accuracy: 0.9462950373895309


In [14]:
df_test = pd.read_json('J:/kaggle/iceberg/test.json')
Xtest = (get_scaled_imgs(df_test))
pred_test = model.predict(Xtest)

In [15]:
submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': pred_test.reshape((pred_test.shape[0]))})
print(submission.head(10))

         id  is_iceberg
0  5941774d    0.366425
1  4023181e    0.904179
2  b20200e4    0.025609
3  e7f018bb    0.995582
4  4371c8c3    0.039017
5  a8d9b1fd    0.001022
6  29e7727e    0.054846
7  92a51ffb    0.996019
8  c769ac97    0.000066
9  aee0547d    0.000002


In [16]:
#save submission
submission.to_csv('J:/kaggle/iceberg/submission.csv', index=False) 